## Business understanding
Water is a basic need and right for all human beings and access to clean and safe water remains a significant challenge in Tanzania, a country with a population exceeding 57 million people. While thousands of water points (wells and pumps) have been installed across the nation, not all of them function reliably. Many are partially functional, in need of repair, or completely non-functional. This undermines efforts to provide consistent access to water, particularly in rural areas. Using data on pump type, installation year, geographic location, and other related attributes, we can build a predictive model that classifies the condition of a water well. Such a system would help prioritize repairs, allocate resources effectively, and guide future infrastructure investments.

## Problem statement
The traditional model for water infrastructure management in Tanzania is reactive: a pump is serviced or repaired only after it has completely failed. This approach is inherently inefficient and costly, leading to prolonged periods without access to water for communities and significant expense for emergency repairs

This project is determined to build predictive model that represents a fundamental paradigm shift to proactive and data driven maintenance. and by leveraging historical data and machine learning, the model can anticipate which water points are at risk of failure or are already in a state of disrepair, enabling maintenance crews to intervene before a complete breakdown occurs.

## Objectives
The most critical and actionable objective of this project is to:
* Accurately identify water points that are currently functional but need repair
* Identify patterns in non-functional wells to inform better design, placement, and maintenance of new water points.

## Metrics of success
The success of the model will be evaluated using classification metrics:

* Accuracy
* precision and recall
* F1 score
* Confusion matrix

## Data understanding
The dataset from the Tanzania Ministry of Water provides records of waterpoints across the country. Each record describes the physical, technical, financial, and management aspects of a waterpoint, alongside its operating condition (target variable: functional, needs repair, or non-functional). The features provided can be grouped as follows:

1. Water availability & technical
 * amount_tsh: total static head (amount of water available)
 * gps_height: altitude of the well
 * construction_year: year the waterpoint was built
 * num_private: private codes/indicators (often missing or unused)
 
2. Location & geography
 * longitude, latitude: GPS coordinates
 * basin: geographic water basin
 * subvillage, region, region_code, district_code, lga, ward: administrative/geographic locations
 
3. Funding & installation
 * funder: who funded the well
 * installer: organization that installed the well
 * scheme_name: water scheme name
 * scheme_management: operator of the scheme
 
4. Management & governance

 * management, management_group: how the waterpoint is managed
 * public_meeting: whether a public meeting was held
 * permit: whether the waterpoint is permitted
 * recorded_by: group recording the data
 
5. Water extraction & source

 * extraction_type, extraction_type_group, extraction_type_class: type of pump/extraction system
 * source, source_type, source_class: water source information

6. Water quality & quantity
 * water_quality, quality_group: reported quality of the water
 * quantity, quantity_group: availability of water

7. Payment & financials
 * payment, payment_type: cost of water use

8. Waterpoint details
 * waterpoint_type, waterpoint_type_group: kind of waterpoint
 * wpt_name: waterpoint name (if present)
 * date_recorded: when the record was entered
 * population: number of people depending on the waterpoint

# Data preparation

In [90]:
# importing the relevant libraries
import pandas as pd 



In [91]:
# loading the datasets
trainig_values_df= pd.read_csv("training_set_values.csv")
training_labels_df= pd.read_csv("training_set_labels.csv")
test_values_df= pd.read_csv("test_set_values.csv")



In [92]:
# reading the training set values df
trainig_values_df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [93]:
training_labels_df.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [94]:
test_values_df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [95]:
# printing the shapes all together 
print("Observation:")
print(f"The training set values has {trainig_values_df.shape[0]} records and {trainig_values_df.shape[1]} features")
print(f"The training set values has {training_labels_df.shape[0]} records and {training_labels_df.shape[1]} features")
print(f"The training set values has {test_values_df.shape[0]} records and {test_values_df.shape[1]} features")


Observation:
The training set values has 59400 records and 40 features
The training set values has 59400 records and 2 features
The training set values has 14850 records and 40 features


In [96]:
# checking datatypes
trainig_values_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [97]:
# checking for missing values in training set values df
missing_values= trainig_values_df.isna().sum()

missing_counts= missing_values.sort_values(ascending= False)
missing_counts

scheme_name              28166
scheme_management         3877
installer                 3655
funder                    3635
public_meeting            3334
permit                    3056
subvillage                 371
ward                         0
lga                          0
district_code                0
region_code                  0
region                       0
waterpoint_type_group        0
population                   0
num_private                  0
wpt_name                     0
latitude                     0
longitude                    0
gps_height                   0
date_recorded                0
amount_tsh                   0
basin                        0
recorded_by                  0
waterpoint_type              0
water_quality                0
source_class                 0
source_type                  0
source                       0
quantity_group               0
quantity                     0
quality_group                0
payment_type                 0
construc

Observation:
    
The training_set_values_df columns have no missing values, just a few columns like scheme name, scheme management, installer, funder, public meeting, permit and subvillage with some missing values which can be handled

In [98]:
# checking value counts of the missing columns
missing_cols = trainig_values_df.columns[trainig_values_df.isnull().any()].tolist()

print(f"Columns with missing values: {missing_cols}")

Columns with missing values: ['funder', 'installer', 'subvillage', 'public_meeting', 'scheme_management', 'scheme_name', 'permit']


In [99]:
for col in missing_cols:
    print(f"\nColumn: {col}")
    print(trainig_values_df[col].value_counts(dropna=False))


Column: funder
Government Of Tanzania    9084
NaN                       3635
Danida                    3114
Hesawa                    2202
Rwssp                     1374
                          ... 
Teresa Munyama               1
Sda Church                   1
Camartec                     1
Nyeisa                       1
Mstiiti                      1
Name: funder, Length: 1898, dtype: int64

Column: installer
DWE           17402
NaN            3655
Government     1825
RWE            1206
Commu          1060
              ...  
NAFCO             1
Kajima            1
Active KMK        1
is                1
Word              1
Name: installer, Length: 2146, dtype: int64

Column: subvillage
Madukani        508
Shuleni         506
Majengo         502
Kati            373
NaN             371
               ... 
Kiwere            1
Mapendekezo       1
Korongo Tatu      1
Kavumu            1
Chamjumbe         1
Name: subvillage, Length: 19288, dtype: int64

Column: public_meeting
True     

In [100]:
# filling in missing values in installer column
trainig_values_df['installer'] = trainig_values_df['installer'].str.lower().str.strip() # normalizing text
trainig_values_df["installer"].value_counts()[:20]

dwe                   17405
government             1891
hesawa                 1395
rwe                    1206
commu                  1065
danida                 1050
district council        961
kkkt                    910
0                       777
central government      763
tcrs                    707
world vision            699
danid                   624
ces                     610
community               555
gover                   511
amref                   443
lga                     413
tasaf                   411
wedeco                  398
Name: installer, dtype: int64

In [101]:
# mapping to unify common mispellings
mapping = {
'world vission': 'world vision',
 'world division': 'world vision',
 'would bank': 'world bank',
 'word bank': 'world bank',
 'world banks': 'world bank',
 'word divisio': 'world division',
 'danid': 'danida',
 'governm': 'government',
 'cental government': 'central government',
 'central govt': 'central government',
 'govern': 'government',
 'serikali': 'government',
 'oxfarm': 'oxfam',
 'unisef': 'unicef',
 'commu': 'community',
 'gove': 'government',
 'finw': 'fini water',
 'fw': 'fini water',
 'finwater': 'fini water',
 'finn water': 'fini water',
 'fin water': 'fini water',
 'gover': 'government',
 '0': 'unknown',
 '-': 'unknown',
 'acra': 'accra',
 'centr': 'central government',
 'communit': 'community',
 'comunity': 'community',
 'cartas': 'caritas',
 'conce': 'concern',
 'not known': 'unknown'}

In [109]:
# replacing in the mapped values
trainig_values_df["installer"] = trainig_values_df["installer"].replace(mapping)
test_values_df["installer"] = test_values_df["installer"].replace(mapping)

In [103]:
# Keep top 20 installers and group the rest into "other".
top_installers = trainig_values_df["installer"].value_counts().nlargest(20).index
trainig_values_df["installer"] = trainig_values_df["installer"].apply(lambda x: x if x in top_installers else "other")
trainig_values_df["installer"].unique().tolist()

['other',
 'world vision',
 'dwe',
 'danida',
 'wedeco',
 'district council',
 'unknown',
 'central government',
 'community',
 'lga',
 'government',
 'kkkt',
 'rwe',
 'hesawa',
 'ces',
 'tasaf',
 'fini water',
 'amref',
 'dmdd',
 'tcrs',
 'norad']

### cleaning funder column


In [104]:
test_values_df["funder"].value_counts()[:20]

Government Of Tanzania    2215
Danida                     793
Hesawa                     580
World Bank                 352
Kkkt                       336
Rwssp                      329
World Vision               316
Unicef                     267
Tasaf                      259
Dhv                        236
Private Individual         208
0                          203
Dwsp                       201
District Council           194
Norad                      184
Water                      156
Germany Republi            155
Ministry Of Water          138
Tcrs                       133
Hifab                      127
Name: funder, dtype: int64

In [114]:
trainig_values_df["funder"].unique().tolist()

['roman',
 'grumeti',
 'lottery club',
 'unicef',
 'action in a',
 'mkinga distric coun',
 'dwsp',
 'rwssp',
 'wateraid',
 'isingiro ho',
 'private',
 'danida',
 'world vision',
 'lawatefuka water supply',
 'biore',
 'rudep',
 'hesawa',
 'twe',
 'isf',
 'african development bank',
 'government of tanzania',
 'sobodo',
 'water',
 'private individual',
 'undp',
 nan,
 'not known',
 'kirde',
 'cefa',
 'ces(gmbh)',
 'european union',
 'lga',
 'district council',
 'muwsa',
 'dwe/norad',
 'kkkt_makwale',
 'sawaka',
 'ces (gmbh)',
 'olgilai village community',
 'kkkt',
 'roman catholic',
 'norad',
 'adra',
 'sema',
 'piusi',
 'dwe',
 'rc church',
 'swisland/ mount meru flowers',
 'ifad',
 'swedish',
 'idc',
 'he',
 'isf/tacare',
 'jica',
 'mzee sh',
 'aict',
 'tcrs',
 'kiuma',
 'germany republi',
 'netherlands',
 'ruthe',
 'tulawaka gold mine',
 'nethalan',
 'tasaf',
 'concern world wide',
 'wfp',
 'lips',
 'sida',
 'world bank',
 'tanza',
 '0',
 'sw',
 'shipo',
 'fini water',
 'kanisa',
 'ox

In [107]:
# normalizing values in funder column for both training and testing data

trainig_values_df['funder'] = trainig_values_df['funder'].str.lower().str.strip() # normalizing text
test_values_df['funder'] = test_values_df['funder'].str.lower().str.strip()

In [83]:
# mapping to unify common mispellings
mapping =
    "oxfarm": "oxfam",
    "govt of tanzania": "government of tanzania",
    "tanzania government": "government of tanzania",
    "germany republi": "german republic",
    "gvt": "government of tanzania",
    "world vision": "world vision",
    "vision world": "world vision",
    "roman": "roman catholic"
    "roman catholic": "church",
    "catholic church": "church"


In [84]:
trainig_values_df["funder"] = trainig_values_df["funder"].replace(mapping)
test_values_df['funder'] = test_values_df["funder"].replace(mapping)

In [86]:
# Keep top 20 funders and group the rest into "other".
top_funders = trainig_values_df["funder"].value_counts().nlargest(20).index
trainig_values_df["funder"] = trainig_values_df["funder"].apply(lambda x: x if x in top_funders else "other")

trainig_values_df["funder"].unique().tolist()

['other',
 'danida',
 'world vision',
 'hesawa',
 'lga',
 'district council',
 'kkkt',
 'dwe',
 'tcrs',
 'tasaf',
 'fini water',
 'amref',
 'community',
 'dmdd',
 'government',
 'unknown']

In [87]:
top_funders = test_values_df["funder"].value_counts().nlargest(20).index
test_values_df['funder'] = test_values_df['funder'].apply(lambda x: x if x in top_funders else 'other')
test_values_df["funder"].unique().tolist()

['dmdd',
 'other',
 'world vision',
 'danida',
 'dwe',
 'tasaf',
 'hesawa',
 'kkkt',
 'fini water',
 'lga',
 'tcrs',
 'community',
 'district council',
 'amref',
 'unknown']

In [14]:
# filling in categorical columns with missing values with unknown
# cat_cols = ['scheme_management', 'funder', 'public_meeting', 'permit']
# for col in cat_cols:
#     trainig_values_df[col] =trainig_values_df[col].fillna("Unknown")
#     test_values_df[col]  = test_values_df[col].fillna("Unknown")

In [15]:
# Drop high-missing / low-value columns
trainig_values_df= trainig_values_df.drop(columns=['scheme_name', 'subvillage'])
test_values_df= test_values_df.drop(columns=['scheme_name', 'subvillage'])


In [59]:
# checking for missing columns value counts
# missing_values= trainig_values_df.isna().sum()

# missing_counts= missing_values.sort_values(ascending= False)
# missing_counts

waterpoint_type_group    0
num_private              0
population               0
ward                     0
lga                      0
district_code            0
region_code              0
region                   0
basin                    0
wpt_name                 0
waterpoint_type          0
latitude                 0
longitude                0
installer                0
gps_height               0
funder                   0
date_recorded            0
amount_tsh               0
public_meeting           0
recorded_by              0
scheme_management        0
permit                   0
source_class             0
source_type              0
source                   0
quantity_group           0
quantity                 0
quality_group            0
water_quality            0
payment_type             0
payment                  0
management_group         0
management               0
extraction_type_class    0
extraction_type_group    0
extraction_type          0
construction_year        0
i